In [1]:
# Data Processing
import pandas as pd
import numpy as np

# Visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# System & File Operations
import os
from pathlib import Path

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Test imports
print("=" * 50)
print("LIBRARY VERSIONS")
print("=" * 50)
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Plotly: {plotly.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")
print(f"Seaborn: {sns.__version__}")
print("=" * 50)
print("All libraries loaded successfully!")

LIBRARY VERSIONS
Pandas: 2.3.0
NumPy: 2.2.6
Plotly: 6.5.0
Matplotlib: 3.10.3
Seaborn: 0.13.2
All libraries loaded successfully!


In [2]:
# Check directory structure
print("📁 PROJECT STRUCTURE")
print("=" * 50)

# Get project root
current_dir = Path.cwd()  # notebooks/
base_dir = current_dir.parent  # global-demographics-analysis/

print(f"Current directory: {current_dir}")
print(f"Project root: {base_dir}")

# Define data folders
data_raw = base_dir / 'data' / 'raw'
data_processed = base_dir / 'data' / 'processed'

print(f"\nTarget folders:")
print(f"  Raw data: {data_raw}")
print(f"  Processed data: {data_processed}")

# Check if folders exist
print("\n📋 Folder status:")
if data_raw.exists():
    print("✅ data/raw/ exists")
else:
    print("⚠️  data/raw/ NOT FOUND")

if data_processed.exists():
    print("✅ data/processed/ exists")
else:
    print("⚠️  data/processed/ NOT FOUND")

print("=" * 50)

📁 PROJECT STRUCTURE
Current directory: d:\substack\birth-rate-analysis\global-demographics-analysis\notebooks
Project root: d:\substack\birth-rate-analysis\global-demographics-analysis

Target folders:
  Raw data: d:\substack\birth-rate-analysis\global-demographics-analysis\data\raw
  Processed data: d:\substack\birth-rate-analysis\global-demographics-analysis\data\processed

📋 Folder status:
✅ data/raw/ exists
✅ data/processed/ exists


In [3]:
# World Bank Data URLs
print("🌐 DATA SOURCES")
print("=" * 50)

# Data download URLs
urls = {
    'fertility_rate': 'https://api.worldbank.org/v2/en/indicator/SP.DYN.TFRT.IN?downloadformat=csv',
    'dependency_ratio': 'https://api.worldbank.org/v2/en/indicator/SP.POP.DPND.OL?downloadformat=csv'
}

# File paths
raw_files = {
    'fertility_rate': data_raw / 'fertility_rate.zip',
    'dependency_ratio': data_raw / 'dependency_ratio.zip'
}

print("📊 Datasets to download:")
print(f"  1. Fertility Rate (TFR)")
print(f"  2. Old-Age Dependency Ratio")
print("\n📁 Download location: data/raw/")
print("=" * 50)

🌐 DATA SOURCES
📊 Datasets to download:
  1. Fertility Rate (TFR)
  2. Old-Age Dependency Ratio

📁 Download location: data/raw/


In [4]:
import urllib.request
import zipfile

def download_worldbank_data(url, filename):
    """Download and extract World Bank CSV data"""
    print(f"\n⏳ Downloading: {filename.name}")
    
    try:
        # Download
        urllib.request.urlretrieve(url, filename)
        print(f"✅ Downloaded: {filename}")
        
        # Extract ZIP
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(filename.parent)
        print(f"✅ Extracted to: {filename.parent}")
        
        return True
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

print("✅ Download function ready")

✅ Download function ready


In [5]:
# Download datasets
print("🌍 DOWNLOADING WORLD BANK DATA")
print("=" * 50)

# Download Fertility Rate
success_1 = download_worldbank_data(
    urls['fertility_rate'], 
    raw_files['fertility_rate']
)

# Download Dependency Ratio
success_2 = download_worldbank_data(
    urls['dependency_ratio'], 
    raw_files['dependency_ratio']
)

print("\n" + "=" * 50)
if success_1 and success_2:
    print("🎉 ALL DATA DOWNLOADED SUCCESSFULLY!")
else:
    print("⚠️  Some downloads failed. Check errors above.")
print("=" * 50)

🌍 DOWNLOADING WORLD BANK DATA

⏳ Downloading: fertility_rate.zip
✅ Downloaded: d:\substack\birth-rate-analysis\global-demographics-analysis\data\raw\fertility_rate.zip
✅ Extracted to: d:\substack\birth-rate-analysis\global-demographics-analysis\data\raw

⏳ Downloading: dependency_ratio.zip
✅ Downloaded: d:\substack\birth-rate-analysis\global-demographics-analysis\data\raw\dependency_ratio.zip
✅ Extracted to: d:\substack\birth-rate-analysis\global-demographics-analysis\data\raw

🎉 ALL DATA DOWNLOADED SUCCESSFULLY!


In [6]:
# List downloaded files
print("📂 FILES IN data/raw/")
print("=" * 50)

raw_files_list = list(data_raw.glob('*'))
raw_files_list.sort()

for file in raw_files_list:
    size_mb = file.stat().st_size / (1024 * 1024)
    print(f"  📄 {file.name:<50} ({size_mb:.2f} MB)")

print("=" * 50)
print(f"✅ Total files: {len(raw_files_list)}")

📂 FILES IN data/raw/
  📄 API_SP.DYN.TFRT.IN_DS2_en_csv_v2_230.csv           (0.18 MB)
  📄 API_SP.POP.DPND.OL_DS2_en_csv_v2_2479.csv          (0.34 MB)
  📄 dependency_ratio.zip                               (0.15 MB)
  📄 fertility_rate.zip                                 (0.07 MB)
  📄 Metadata_Country_API_SP.DYN.TFRT.IN_DS2_en_csv_v2_230.csv (0.06 MB)
  📄 Metadata_Country_API_SP.POP.DPND.OL_DS2_en_csv_v2_2479.csv (0.06 MB)
  📄 Metadata_Indicator_API_SP.DYN.TFRT.IN_DS2_en_csv_v2_230.csv (0.00 MB)
  📄 Metadata_Indicator_API_SP.POP.DPND.OL_DS2_en_csv_v2_2479.csv (0.00 MB)
✅ Total files: 8


In [7]:
# Find main CSV files (not metadata)
print("\n🎯 MAIN DATA FILES")
print("=" * 50)

# Fertility Rate CSV
fertility_csv = list(data_raw.glob('API_SP.DYN.TFRT.IN*.csv'))[0]
print(f"✅ Fertility Rate CSV: {fertility_csv.name}")

# Dependency Ratio CSV
dependency_csv = list(data_raw.glob('API_SP.POP.DPND.OL*.csv'))[0]
print(f"✅ Dependency Ratio CSV: {dependency_csv.name}")

print("=" * 50)


🎯 MAIN DATA FILES
✅ Fertility Rate CSV: API_SP.DYN.TFRT.IN_DS2_en_csv_v2_230.csv
✅ Dependency Ratio CSV: API_SP.POP.DPND.OL_DS2_en_csv_v2_2479.csv


In [8]:
# Load Fertility Rate data
print("📊 LOADING FERTILITY RATE DATA")
print("=" * 50)

# Read CSV (skip first 4 rows - World Bank format)
df_fertility = pd.read_csv(fertility_csv, skiprows=4)

print(f"✅ Loaded: {fertility_csv.name}")
print(f"📏 Shape: {df_fertility.shape[0]} rows × {df_fertility.shape[1]} columns")
print("\n🔍 First 5 rows:")
print(df_fertility.head())

📊 LOADING FERTILITY RATE DATA
✅ Loaded: API_SP.DYN.TFRT.IN_DS2_en_csv_v2_230.csv
📏 Shape: 266 rows × 70 columns

🔍 First 5 rows:
                  Country Name Country Code  \
0                        Aruba          ABW   
1  Africa Eastern and Southern          AFE   
2                  Afghanistan          AFG   
3   Africa Western and Central          AFW   
4                       Angola          AGO   

                             Indicator Name  Indicator Code      1960  \
0  Fertility rate, total (births per woman)  SP.DYN.TFRT.IN  4.567000   
1  Fertility rate, total (births per woman)  SP.DYN.TFRT.IN  6.650310   
2  Fertility rate, total (births per woman)  SP.DYN.TFRT.IN  7.282000   
3  Fertility rate, total (births per woman)  SP.DYN.TFRT.IN  6.468887   
4  Fertility rate, total (births per woman)  SP.DYN.TFRT.IN  6.708000   

       1961      1962      1963      1964      1965      1966      1967  \
0  4.422000  4.262000  4.107000  3.940000  3.797000  3.621000  3.452000   

In [9]:
# Inspect data structure
print("Data Shape:", df_fertility.shape)
print("\nColumn Names:")
print(df_fertility.columns.tolist()[:10], "... (first 10)")

print("\nData Types:")
print(df_fertility.dtypes.head(10))

print("\nMissing Values in Key Columns:")
print(df_fertility[['Country Name', 'Country Code', '2023']].isnull().sum())

print("\nUnique Countries:", df_fertility['Country Name'].nunique())

Data Shape: (266, 70)

Column Names:
['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965'] ... (first 10)

Data Types:
Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
1960              float64
1961              float64
1962              float64
1963              float64
1964              float64
1965              float64
dtype: object

Missing Values in Key Columns:
Country Name    0
Country Code    0
2023            1
dtype: int64

Unique Countries: 266


In [10]:
# Transform from wide to long format
print("Transforming data: Wide -> Long format\n")

# Select relevant columns (Country info + year columns)
year_columns = [col for col in df_fertility.columns if col.isdigit()]
id_columns = ['Country Name', 'Country Code']

# Melt the dataframe
df_fertility_long = df_fertility[id_columns + year_columns].melt(
    id_vars=id_columns,
    var_name='Year',
    value_name='Fertility_Rate'
)

# Convert Year to integer
df_fertility_long['Year'] = df_fertility_long['Year'].astype(int)

# Remove missing values
df_fertility_long = df_fertility_long.dropna(subset=['Fertility_Rate'])

print("Transformation complete")
print("New shape:", df_fertility_long.shape)
print("\nFirst 10 rows:")
print(df_fertility_long.head(10))
print("\nLast 10 rows:")
print(df_fertility_long.tail(10))

Transforming data: Wide -> Long format

Transformation complete
New shape: (16928, 4)

First 10 rows:
                  Country Name Country Code  Year  Fertility_Rate
0                        Aruba          ABW  1960        4.567000
1  Africa Eastern and Southern          AFE  1960        6.650310
2                  Afghanistan          AFG  1960        7.282000
3   Africa Western and Central          AFW  1960        6.468887
4                       Angola          AGO  1960        6.708000
5                      Albania          ALB  1960        6.383000
6                      Andorra          AND  1960        2.545000
7                   Arab World          ARB  1960        6.922033
8         United Arab Emirates          ARE  1960        6.499000
9                    Argentina          ARG  1960        3.136000

Last 10 rows:
                Country Name Country Code  Year  Fertility_Rate
17014  Virgin Islands (U.S.)          VIR  2023        1.980000
17015               Viet Nam 

In [11]:
# Load Dependency Ratio data
df_dependency = pd.read_csv(dependency_csv, skiprows=4)

print("Original shape:", df_dependency.shape)

# Transform to long format
year_columns = [col for col in df_dependency.columns if col.isdigit()]
id_columns = ['Country Name', 'Country Code']

df_dependency_long = df_dependency[id_columns + year_columns].melt(
    id_vars=id_columns,
    var_name='Year',
    value_name='Old_Age_Dependency_Ratio'
)

df_dependency_long['Year'] = df_dependency_long['Year'].astype(int)
df_dependency_long = df_dependency_long.dropna(subset=['Old_Age_Dependency_Ratio'])

print("Transformed shape:", df_dependency_long.shape)
print("\nFirst 5 rows:")
print(df_dependency_long.head())

Original shape: (266, 70)
Transformed shape: (17195, 4)

First 5 rows:
                  Country Name Country Code  Year  Old_Age_Dependency_Ratio
0                        Aruba          ABW  1960                  5.229128
1  Africa Eastern and Southern          AFE  1960                  5.631545
2                  Afghanistan          AFG  1960                  5.112019
3   Africa Western and Central          AFW  1960                  6.009687
4                       Angola          AGO  1960                  5.678100


In [12]:
# Filter data for 2023 (most recent year)
df_map = df_fertility_long[df_fertility_long['Year'] == 2023].copy()

print("Data for World Map (2023)")
print("Shape:", df_map.shape)
print("\nFirst 10 countries:")
print(df_map.head(10))

print("\nFertility Rate statistics:")
print(df_map['Fertility_Rate'].describe())

print("\nHighest fertility:")
print(df_map.nlargest(5, 'Fertility_Rate')[['Country Name', 'Fertility_Rate']])

print("\nLowest fertility:")
print(df_map.nsmallest(5, 'Fertility_Rate')[['Country Name', 'Fertility_Rate']])

Data for World Map (2023)
Shape: (265, 4)

First 10 countries:
                      Country Name Country Code  Year  Fertility_Rate
16758                        Aruba          ABW  2023        1.602000
16759  Africa Eastern and Southern          AFE  2023        4.223820
16760                  Afghanistan          AFG  2023        4.840000
16761   Africa Western and Central          AFW  2023        4.497707
16762                       Angola          AGO  2023        5.124000
16763                      Albania          ALB  2023        1.348000
16764                      Andorra          AND  2023        1.082000
16765                   Arab World          ARB  2023        3.089751
16766         United Arab Emirates          ARE  2023        1.200000
16767                    Argentina          ARG  2023        1.500000

Fertility Rate statistics:
count    265.000000
mean       2.411560
std        1.194905
min        0.586000
25%        1.490000
50%        1.980000
75%        3.129000

In [13]:
# Create world map with Plotly
fig = px.choropleth(
    df_map,
    locations='Country Code',
    locationmode='ISO-3',
    color='Fertility_Rate',
    hover_name='Country Name',
    hover_data={'Country Code': False, 'Year': False, 'Fertility_Rate': ':.2f'},
    color_continuous_scale=[
        [0.0, '#08519c'],   # Dark blue (low)
        [0.3, '#3182bd'],   # Medium blue
        [0.4, '#6baed6'],   # Light blue
        [0.47, '#c6dbef'],  # Very light blue
        [0.5, '#fff7bc'],   # Pale yellow (replacement level ~2.1)
        [0.6, '#fee391'],   # Light yellow
        [0.7, '#fec44f'],   # Yellow-orange
        [0.85, '#fe9929'],  # Orange
        [1.0, '#d95f0e']    # Dark orange/red (high)
    ],
    range_color=[0.5, 7],
    labels={'Fertility_Rate': 'Fertility Rate<br>(children per woman)'},
    title='Global Fertility Rates (2023): A Divided World'
)

# Update layout
fig.update_layout(
    width=750,
    height=450,
    font=dict(size=12),
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    coloraxis_colorbar=dict(
        title="Fertility Rate",
        tickvals=[1, 2.1, 3, 4, 5, 6],
        ticktext=['1.0', '2.1<br>(Replacement)', '3.0', '4.0', '5.0', '6.0+']
    )
)

fig.show()

In [14]:
# Save the map to visualizations folder
output_path = base_dir / 'visualizations' / 'fertility_map_2023.png'

# Save as PNG (high resolution)
fig.write_image(
    output_path,
    width=1200,
    height=700,
    scale=2  # 2x resolution for better quality
)

print(f"Map saved to: {output_path}")

Map saved to: d:\substack\birth-rate-analysis\global-demographics-analysis\visualizations\fertility_map_2023.png


In [15]:
# Load dependency ratio data for selected countries
selected_countries = {
    'JPN': 'Japan',
    'ITA': 'Italy', 
    'DEU': 'Germany',
    'KOR': 'South Korea',
    'HUN': 'Hungary'
}

# Filter data for selected countries and years 1980-2024
df_dep_chart = df_dependency_long[
    (df_dependency_long['Country Code'].isin(selected_countries.keys())) &
    (df_dependency_long['Year'] >= 1980) &
    (df_dependency_long['Year'] <= 2024)
].copy()

# Map country codes to readable names
df_dep_chart['Country'] = df_dep_chart['Country Code'].map(selected_countries)

print("Data shape:", df_dep_chart.shape)
print("\nCountries included:")
print(df_dep_chart.groupby('Country')['Year'].agg(['min', 'max', 'count']))

Data shape: (225, 5)

Countries included:
              min   max  count
Country                       
Germany      1980  2024     45
Hungary      1980  2024     45
Italy        1980  2024     45
Japan        1980  2024     45
South Korea  1980  2024     45


In [18]:
# Create line chart for dependency ratio
fig2 = px.line(
    df_dep_chart,
    x='Year',
    y='Old_Age_Dependency_Ratio',
    color='Country',
    markers=True,
    title='Old-Age Dependency Ratio (1980-2024): The Ponzi Scheme Unraveling',
    labels={
        'Old_Age_Dependency_Ratio': 'Old-Age Dependency Ratio<br>(65+ per 100 working-age)',
        'Year': 'Year'
    }
)

fig2.update_layout(
    width=750,
    height=450,
    font=dict(size=12),
    hovermode='x unified',
    legend=dict(
        title='Country',
        orientation='v',
        yanchor='top',
        y=0.95,
        xanchor='left',
        x=0.02
    ),
    xaxis=dict(dtick=5),
    yaxis=dict(range=[0, 60])
)

# Add annotation for replacement context
fig2.add_annotation(
    x=2000,
    y=50,
    text="Higher ratio = More retirees per worker<br>System strain increases",
    showarrow=False,
    font=dict(size=10, color='gray'),
    align='left'
)

fig2.show()

In [ ]:
# Save dependency ratio chart
output_dep_png = base_dir / 'visualizations' / 'dependency_ratio_1980_2024.png'

fig2.write_image(output_dep_png, width=1200, height=600, scale=2)
fig2.write_html(output_dep_html)

print(f"Chart saved:")
print(f"  PNG: {output_dep_png}")

Chart saved:
  PNG: d:\substack\birth-rate-analysis\global-demographics-analysis\visualizations\dependency_ratio_1980_2024.png
  HTML: d:\substack\birth-rate-analysis\global-demographics-analysis\visualizations\dependency_ratio_1980_2024.html


In [20]:
# Filter South Korea fertility data
df_korea = df_fertility_long[
    (df_fertility_long['Country Code'] == 'KOR') &
    (df_fertility_long['Year'] >= 2006) &
    (df_fertility_long['Year'] <= 2024)
].copy()

print("South Korea TFR (2006-2024)")
print(df_korea[['Year', 'Fertility_Rate']])

print("\nKey statistics:")
print(f"2006: {df_korea[df_korea['Year']==2006]['Fertility_Rate'].values[0]:.3f}")
print(f"2023: {df_korea[df_korea['Year']==2023]['Fertility_Rate'].values[0]:.3f}")
print(f"Change: {df_korea[df_korea['Year']==2023]['Fertility_Rate'].values[0] - df_korea[df_korea['Year']==2006]['Fertility_Rate'].values[0]:.3f}")

South Korea TFR (2006-2024)
       Year  Fertility_Rate
12362  2006           1.132
12628  2007           1.259
12894  2008           1.192
13160  2009           1.149
13426  2010           1.226
13692  2011           1.244
13958  2012           1.297
14224  2013           1.187
14490  2014           1.205
14756  2015           1.239
15022  2016           1.172
15288  2017           1.052
15554  2018           0.977
15820  2019           0.918
16086  2020           0.837
16352  2021           0.808
16618  2022           0.778
16884  2023           0.721

Key statistics:
2006: 1.132
2023: 0.721
Change: -0.411


In [25]:
# Create South Korea TFR chart
fig3 = px.line(
    df_korea,
    x='Year',
    y='Fertility_Rate',
    markers=True,
    title='South Korea: $211 Billion Spent, Fertility Still Collapsing',
    labels={
        'Fertility_Rate': 'Total Fertility Rate<br>(children per woman)',
        'Year': 'Year'
    }
)

fig3.update_layout(
    width=750,
    height=450,
    font=dict(size=12),
    showlegend=False
)

# Add horizontal line for replacement level
fig3.add_hline(
    y=2.1, 
    line_dash="dash", 
    line_color="red",
    annotation_text="Replacement Level (2.1)",
    annotation_position="right"
)

# Add spending annotations
fig3.add_annotation(
    x=2021,
    y=1.7,
    text="<b>Total Pro-Natalist Spending (2006-2024):</b><br>$211 billion USD<br><br><b>2024 Annual Budget:</b><br>$30 billion USD<br><br><b>Result:</b><br>TFR fell from 1.13 to 0.72<br>(36% decline)",
    showarrow=False,
    font=dict(size=8),
    align='left',
    bordercolor='black',
    borderwidth=1,
    borderpad=10,
    bgcolor='lightyellow',
    opacity=0.9
)

# Highlight 2023 point
fig3.add_annotation(
    x=2023,
    y=0.721,
    text="0.72<br>(World's lowest)",
    showarrow=True,
    arrowhead=2,
    ax=-40,
    ay=-40,
    font=dict(size=10, color='red')
)

fig3.show()

In [ ]:
# Save South Korea TFR chart
output_korea_png = base_dir / 'visualizations' / 'south_korea_tfr_spending.png'

fig3.write_image(output_korea_png, width=1200, height=600, scale=2)
fig3.write_html(output_korea_html)

print(f"Chart saved:")
print(f"  PNG: {output_korea_png}")

Chart saved:
  PNG: d:\substack\birth-rate-analysis\global-demographics-analysis\visualizations\south_korea_tfr_spending.png
  HTML: d:\substack\birth-rate-analysis\global-demographics-analysis\visualizations\south_korea_tfr_spending.html
